In [3]:
# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),            # Resize images to match the input size of the pre-trained model
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3-channel
    transforms.ToTensor(),                     # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the CNN Model with Transfer Learning
class TransferLearningModel(nn.Module):
    def __init__(self):
        super(TransferLearningModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)  # Load a pre-trained ResNet model
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 10)  # Modify the final layer for 10 classes (digits 0-9)

    def forward(self, x):
        return self.base_model(x)

# Initialize the model, loss function, and optimizer
model = TransferLearningModel().to(device)  # Move model to GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the Model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")

# Evaluate the Model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test dataset: {100 * correct / total:.2f}%')

# Visualize Results (Optional)
# You can include code here to visualize predictions or training metrics.


Epoch [1/5], Loss: 0.0677
Epoch [2/5], Loss: 0.0347
Epoch [3/5], Loss: 0.0288
Epoch [4/5], Loss: 0.0231
Epoch [5/5], Loss: 0.0200
Accuracy of the model on the test dataset: 98.90%
